<a href="https://colab.research.google.com/github/louisdennington-design/decision-tree-dissertation/blob/main/llm_makes_json.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# Mount Google Drive

from google.colab import drive
drive.mount('/content/drive', force_remount = True)

ValueError: mount failed

In [2]:
import os
import json
from transformers import AutoModelForCausalLM, AutoTokenizer

In [ ]:
# Set base parameters

MODEL_NAME = "Qwen/Qwen2.5-7B-Instruct"

LOAD_PATH = "/content/drive/My Drive/Colab Notebooks/Dissertation/Scrapes"
LOAD_FILE = os.path.join(LOAD_PATH, "guideline_raw.json")

SAVE_PATH = "/content/drive/My Drive/Colab Notebooks/Dissertation/JSON"
os.makedirs(SAVE_PATH, exist_ok=True)
SAVE_FILE = os.path.join(SAVE_PATH, "guideline_structured.json")

In [ ]:
# Load LLM

"""
Focus should be on instruction-following models from Hugging Face
With free licence (Apache)
Qwen seems to have been trained on producing JSON formats
...allows for many tokens as input
...parameters are good balance between small and big
Should also check Llama offerings?
"""

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype="auto",
    device_map="auto")

In [ ]:
# Test

## Should also carry out test prompt of transforming recommendations

text = "Should someone with a diagnosis of bipolar who is taking lithium be referred to secondary care if they are mildly irritable?"

inputs = tokenizer(text, return_tensors="pt").to(model.device)

outputs = model.generate(**inputs, max_new_tokens=500)

response = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(response)

In [1]:
# Load JSON of raw recommendations

def load_json():
    try:
        with open(LOAD_FILE, "r", encoding="utf-8") as raw_recommendations:
            return json.load(raw_recommendations)
    except FileNotFoundError:
        raise FileNotFoundError(f'JSON file not found: {LOAD_FILE}')

raw_recommendations = load_json()

print(type(raw_recommendations))
print(len(raw_recommendations))
print(raw_recommendations[0])

NameError: name 'LOAD_FILE' is not defined

In [ ]:
def construct_prompt(entity):

    """
    Given one recommendation entry {}, creates the prompt to extract one normalised JSON item
    """

    heading_1 = entity.get('heading_1')
    sub_heading_1 = entity.get('sub_heading_1')
    sub_heading_2 = entity.get('sub_heading_2')

    original_recommendation_number = entity.get('original_recommendation_number')
    original_recommendation_text = entity.get('original_recommendation_text')

    heading_context = " > ".join(h.strip() for h in [heading_1, sub_heading_1, sub_heading_2] if isinstance(h, str) and h.strip())

    return f"""
    You are extracting structured information from a NICE guideline recommendation.

    RULES:
    - output must be valid JSON only (no markdown)
    - do not invent clinical information, thresholds or populations; use only what is present in the recommendation text
    - For ALL clinical descriptor fields (e.g. phase, severity, medication): populate a value ONLY if it is explicitly stated in the recommendation text. Do NOT infer information that is not directly stated. If not explicit, use null.
    - 'action' must be a verb phrase indicating what is being done and to/for whom (e.g., 'ensure that people have access to calming environments and reduced stimulation', not just 'ensure')
    - 'scope' must be the setting/service/context (e.g., 'in primary care', 'in secondary care', 'in a service that can…', 'when assessing…')
    - 'population' is the group the recommendation applies to (e.g., 'people with…', 'older people…', 'adults…', 'children…', 'pregnant women…', etc.)
    - Extract 'conditionality' from clauses that begin 'if...' or 'where...'
    - Extract 'prohibitions' from verb phrases including 'do not', 'must not' or 'should not'
    - 'urgency' as 'True' if the text includes 'urgent', 'urgently', 'immediate' or 'immediately', otherwise 'False'
    - 'episode_history' must be one of: ['none', 'one', 'multiple', null]
    - 'phase' must be one of: ['mania', 'hypomania', 'bipolar_depression', 'mixed', 'rapid_cycling', 'euthymic', null]
    - 'severity' must be one of: ['mild', 'moderate', 'severe', null]
    - 'current_psychosis' must be: ['present', 'absent', null]
    - 'diagnoses' must be one or more comorbid mental health diagnoses. If more than one diagnosis is mentioned, record all as a list of strings.
    - 'current_medication' must be a medication name or null
    - 'adherence' must be one of: ['good', 'poor', null]
    - 'physical_health' must be the name of a physical disease. If more than one diagnosis is mentioned, record all as a list of strings.
    - 'risk' must be one of: ['self_harm', 'risk_to_others', null]
    - 'psychological_therapy' must be one of: ['offered', null]
    - 'care coordination' must be one of: ['current', 'offered', null]
    - you MUST use 'null' if the information for any field is not explicit in the recommendation or heading
    - if there is more than one value for any field, retain all as a list of strings

    CONTEXT: {heading_context}

    RECOMMENDATION NUMBER: {original_recommendation_number}
    RECOMMENDATION TEXT: {original_recommendation_text}

    Produce JSON with exactly these keys:
    - action
    - scope
    - population
    - conditionality
    - prohibitions
    - urgency
    - episode_history
    - phase
    - severity
    - current_psychosis
    - diagnoses
    - current_medication
    - adherence
    - physical_health
    - risk
    - psychological_therapy
    - care_coordination
    - original_recommendation_number
    - original_recommendation_text
    """


In [ ]:
# Check prompt length

recommendation_for_prompt_check = raw_recommendations[123]

prompt = construct_prompt(recommendation_for_prompt_check)

In [ ]:
def run_llm_on_entity(tokenizer, model, entity):

    """
    Call the model on a single prompt using the prompt function
    Return model response
    """

    prompt = construct_prompt(entity)

    inputs = tokenizer(prompt,
                       return_tensors="pt").to(model.device)

    outputs = model.generate(**inputs,
                             max_new_tokens=500,
                             do_sample=False) # deterministic decoding without random sampling
                                            # if removed, reinstate temperature / top_p / top_k

    llm_response = tokenizer.batch_decode(outputs[:, inputs["input_ids"].shape[1]:],
                                          skip_special_tokens=True)

    return llm_response[0]

In [ ]:
def convert_output_to_true_json(llm_response):
    """
    Takes output from run_llm_on_entity
    Turns it into a true JSON dictionary
    """

    llm_response = llm_response.strip()

    start = llm_response.find("{")
    end = llm_response.rfind("}")

    if start == -1 or end == -1 or end < start: # Where -1 is "not found" for str.find()
        raise ValueError("Could not find a JSON object in the LLM output.")

    json_string = llm_response[start:end + 1].strip()

    try:
        json_object = json.loads(json_string)
    except json.JSONDecodeError as e:
        print(f"JSON parsing error: {e}")
        raise

    if not isinstance(json_object, dict):
        raise TypeError(f"The object created by the function convert_output_to_true_json is not a JSON object. Instead it is a {type(json_object)}.")

    return json_object

In [ ]:
def validate_json(json_object, required_keys):

    missing_keys = [k for k in required_keys if k not in json_object]
    if missing_keys:
        raise ValueError(f"Missing required keys: {missing_keys}")

    extra_keys = [k for k in json_object.keys() if k not in required_keys]
    if extra_keys:
        raise ValueError(f"Unexpected extra keys: {extra_keys}")

    for key, value in json_object.items():
        if value is None:
            continue
        if key == "urgency" and isinstance(value, bool):
            continue
        if isinstance(value, str):
            continue
        raise TypeError(f"Key '{key}' is of the wrong type, namely: {type(value)}.")

    return json_object

In [ ]:
def orchestrate_create_json(raw_recommendations, tokenizer, model, save_file):

    compiled_recommendations = []

    errors = []

    required_keys = ['action'
                    'scope'
                    'population'
                    'conditionality'
                    'prohibitions'
                    'urgency'
                    'episode_history'
                    'phase'
                    'severity'
                    'current_psychosis'
                    'diagnoses'
                    'current_medication '
                    'adherence'
                    'physical_health'
                    'risk'
                    'psychological_therapy'
                    'care_coordination'
                    'original_recommendation_number'
                    'original_recommendation_text']

    for i, entity in enumerate(raw_recommendations[87:92]): # Remove numbers to process full batch

        counter = 0

        llm_output_text = run_llm_on_entity(tokenizer, model, entity)

        try:
            parsed_json = convert_output_to_true_json(llm_output_text)
            validate_json(parsed_json, required_keys)

        except Exception as e:
            print(f"Error {e} at point {i}")
            errors.append({"index": i, "error": str(e), "raw_llm_output": llm_output_text})
            continue

        compiled_recommendations.append(parsed_json)

        counter += 1

        print(f'Number of recommendations processed: {counter}')

    with open(save_file, "w", encoding="utf-8") as f:
        json.dump(compiled_recommendations, f, ensure_ascii=False, indent=2)

    print(f"Here is the list of json parsing errors: {errors}")

    return compiled_recommendations, errors

In [ ]:
orchestrate_create_json(raw_recommendations, tokenizer, model, SAVE_FILE)